In [1]:
%cd /workspace
import sys
import os

# Set PYTHONPATH environment variable for the kernel
robofin_path = os.path.join(os.getcwd(), 'robofin')
current_pythonpath = os.environ.get('PYTHONPATH', '')
if robofin_path not in current_pythonpath:
    os.environ['PYTHONPATH'] = f"{robofin_path}:{current_pythonpath}" if current_pythonpath else robofin_path

# Also add to sys.path for immediate effect
if robofin_path not in sys.path:
    sys.path.insert(0, robofin_path)

/workspace


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import torch
from robofin.robots import Robot

from robofin.robots_original import FrankaRobot
from robofin.robot_constants import FrankaConstants

neutral_config = FrankaConstants.NEUTRAL

# Load the Robot class with the standard URDF file (uses relative paths)
robot = Robot("assets/panda/panda.urdf")

In [ ]:
robot.fixed_eef_link_transforms["panda_hand"]

array([[-1.00000000e+00,  2.79315693e-12,  0.00000000e+00,
         0.00000000e+00],
       [-2.79319215e-12, -1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        -1.00000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

: 

In [4]:
# test eef_fk

robot.eef_fk(np.eye(4))["panda_leftfinger"]

----------------------------------------------------------------------------------------------------
parent (panda_hand) pose:
 [[-1.00000000e+00  2.79315693e-12  0.00000000e+00  0.00000000e+00]
 [-2.79319215e-12 -1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -1.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
joint (panda_finger_joint1) transform:
 [[1.     0.     0.     0.    ]
 [0.     1.     0.     0.04  ]
 [0.     0.     1.     0.0584]
 [0.     0.     0.     1.    ]]
child (panda_leftfinger) pose:
 [[-1.00000000e+00  2.79315693e-12  0.00000000e+00  0.00000000e+00]
 [-2.79319215e-12 -1.00000000e+00  0.00000000e+00  4.00000000e-02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -4.16000000e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
----------------------------------------------------------------------------------------------------
parent (panda_hand) pose:
 [[-1.0000

array([[-1.00000000e+00,  2.79315693e-12,  0.00000000e+00,
         0.00000000e+00],
       [-2.79319215e-12, -1.00000000e+00,  0.00000000e+00,
         4.00000000e-02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        -4.16000000e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
import numpy as np
import torch
from typing import Dict

np_configs: np.ndarray = np.array([neutral_config, neutral_config])
torch_configs: torch.Tensor = torch.tensor(np_configs)

print(np_configs.shape)
print(torch_configs.shape)

In [ ]:
fk_np: Dict[str, np.ndarray] = robot.fk(neutral_config)
fk_torch: Dict[str, torch.Tensor] = robot.fk_torch(torch.tensor(neutral_config))

In [ ]:
np_visual_fk = robot.visual_fk(np_configs)
torch_visual_fk = robot.visual_fk_torch(torch_configs)

In [ ]:
print(fk_np.keys())
print(fk_torch.keys())
fk_torch["right_gripper"]

print(np_visual_fk.keys())
print(torch_visual_fk.keys())

In [ ]:

np_fk_gripper = fk_np["right_gripper"]
torch_fk_gripper = fk_torch["right_gripper"].detach().cpu().numpy()

print("Numpy FK shape:", np_fk_gripper.shape)
print("Torch FK shape:", torch_fk_gripper.shape)
print("Difference (max abs):", np.abs(np_fk_gripper - torch_fk_gripper).max())

# Optionally, test visual FK as well
if hasattr(robot, "visual_fk") and hasattr(robot, "visual_fk_torch"):
    np_visual_fk = robot.visual_fk(np_configs)
    torch_visual_fk = robot.visual_fk_torch(torch_configs)
    np_vis_gripper = np_visual_fk["panda_hand"]
    torch_vis_gripper = torch_visual_fk["panda_hand"].detach().cpu().numpy()
    print("Visual FK difference (max abs):", np.abs(np_vis_gripper - torch_vis_gripper).max())

In [ ]:
np_visual_fk = robot.visual_fk(np_configs, link_name="panda_hand")
torch_visual_fk = robot.visual_fk_torch(torch_configs, link_name="panda_hand")

In [ ]:

from robofin.samplers import NumpyRobotSampler
from robofin.samplers_original import NumpyFrankaSampler

franka_visual_fk = franka_arm_visual_fk(neutral_config, 0.02, np.eye(4))

sampler = NumpyRobotSampler(robot=robot, num_robot_points=1000, num_eef_points=100, use_cache=True)
sampler.sample(neutral_config, 0.02, num_points=100)